In [11]:
import pandas as pd
import featuretools as ft

In [20]:
clients = pd.read_csv('data/clients.csv', parse_dates=['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates=['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates=['payment_date'])

In [21]:
clients.head(1)

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527


In [22]:
loans.head(1)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15


In [23]:
payments.head(1)

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1


## Create new entityset

In [10]:
es = ft.EntitySet(id='clients')

## Add entities

In [34]:
es = es.entity_from_dataframe(entity_id='clients', dataframe=clients,
                              index='client_id', time_index='joined')
es = es.entity_from_dataframe(entity_id='loans', dataframe=loans,
                              variable_types={'repaid': ft.variable_types.Categorical},
                              index='loan_id', time_index='loan_start')
es = es.entity_from_dataframe(entity_id='payments', dataframe=payments,
                              variable_types={'missed': ft.variable_types.Categorical},
                              #make_index=True,
                              index='payment_id',
                              time_index='payment_date')

In [35]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 5]
    loans [Rows: 443, Columns: 9]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [36]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    clients_id (dtype: numeric)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 5)

In [37]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    loans_id (dtype: numeric)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 9)

In [39]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

## Relationship

In [40]:
r_clients = ft.Relationship(es['clients']['client_id'], es['loans']['client_id'])
es = es.add_relationship(r_clients)
r_payments = ft.Relationship(es['loans']['loan_id'], es['payments']['loan_id'])
es = es.add_relationship(r_payments)

In [41]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 5]
    loans [Rows: 443, Columns: 9]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

## Feature Primitives

In [55]:
primitives = ft.list_primitives()
primitives[primitives['type']=='aggregation']

,name,type,description
0,all,aggregation,Test if all values are 'True'.
1,count,aggregation,Counts the number of non null values.
2,last,aggregation,Returns the last value.
3,skew,aggregation,Computes the skewness of a data set.
4,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
5,mode,aggregation,Finds the most common element in a categorical feature.
6,num_unique,aggregation,Returns the number of unique categorical variables.
7,time_since_last,aggregation,Time since last related instance.
8,sum,aggregation,Sums elements of a numeric or boolean feature.
9,median,aggregation,Finds the median value of any feature with well-ordered values.


In [54]:
primitives[primitives['type']=='transform']

,name,type,description
19,characters,transform,Return the characters in a given string.
20,hour,transform,Transform a Datetime feature into the hour.
21,and,transform,"For two boolean values, determine if both values are 'True'."
22,latitude,transform,Returns the first value of the tuple base feature.
23,weekday,transform,Transform Datetime feature into the boolean of Weekday.
24,is_null,transform,"For each value of base feature, return 'True' if value is null."
25,days_since,transform,"For each value of the base feature, compute the number of days between it"
26,haversine,transform,Calculate the approximate haversine distance in miles between two LatLong variable types.
27,cum_min,transform,Calculates the min of previous values of an instance for each value in a time-dependent entity.
28,weekend,transform,Transform Datetime feature into the boolean of Weekend.


In [64]:
features, feature_names = ft.dfs(entityset=es, target_entity='clients',
                                        agg_primitives=['mean', 'max', 'percent_true', 'last'],
                                        trans_primitives=['years', 'month', 'subtract', 'divide'])

In [65]:
clients.head(1)

,clients_id,client_id,joined,income,credit_score
0,0,46109,2002-04-16,172677,527


In [66]:
features.head(1)

,clients_id,income,credit_score,MEAN(loans.loans_id),MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loans_id),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loans_id),...,MAX(loans.loan_amount) / MAX(loans.loans_id),income / LAST(payments.payment_amount),LAST(loans.loan_amount) / MEAN(payments.payment_amount),LAST(loans.rate) / MEAN(payments.payment_amount),credit_score - clients_id / income - clients_id,clients_id - income / MAX(payments.payment_amount),LAST(loans.rate) / clients_id,MAX(loans.loan_amount) / credit_score - income,LAST(loans.loans_id) / credit_score - clients_id,income / credit_score - income
client_id,,,,,,,,,,,,,,,,,,,,,
25707,4,211422,621,89.5,7963.95,3.477,99,13913,9.44,83,...,140.535354,877.26971,1.869242,0.006279,0.002918,-78.18713,1.85,-0.066001,0.134522,-1.002946


In [67]:
features['MONTH(joined)'].head()

client_id
25707    10
26326     5
26695     8
26945    11
29841     8
Name: MONTH(joined), dtype: int64

## Deep Feature Synthesis

In [68]:
# Show a feature with a depth of 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
25707,293.500000
26326,977.375000
26695,1769.166667
26945,1598.666667
29841,1125.500000
32726,799.500000
32885,1729.000000
32961,282.600000
35089,110.400000


the LAST(loans.(MEAN(payments.payment_amount))) has depth = 2 because it is made by stacking two feature primitives, first an aggregation and then a transformation. This feature represents the average payment amount for the last (most recent) loan for each client.

## Automated Deep Feature Synthesis

In [69]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', max_depth = 2)

In [70]:
features

,clients_id,income,credit_score,SUM(loans.loans_id),SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loans_id),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loans_id),...,NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,4,211422,621,1790,159279,69.54,5.916080,4149.486062,2.484186,99,...,6,0,27,1,2010,2007,1,8,3,0
26326,10,227920,633,3256,116321,40.28,4.760952,4393.666631,2.057142,211,...,5,0,6,6,2003,2005,4,7,5,2
26695,19,174532,680,6435,140845,44.39,5.338539,4196.462499,1.561659,366,...,6,0,3,14,2003,2005,9,4,1,1
26945,24,214516,806,6525,106889,42.83,4.472136,4543.621769,1.619717,442,...,6,0,16,1,2002,2004,12,5,0,1
29841,13,38354,523,4545,176634,62.01,5.338539,4209.224171,2.122904,261,...,7,1,1,15,2005,2007,3,2,5,1
32726,6,235705,730,2451,126032,58.12,5.627314,4516.148677,1.985567,138,...,7,0,2,1,2004,2004,3,9,2,6
32885,15,58955,642,4335,148806,36.54,4.472136,3205.122414,2.771696,296,...,6,0,13,12,2007,2009,9,6,4,5
32961,12,230341,714,3995,133998,66.82,5.049752,4968.314220,2.753583,243,...,7,0,6,19,2001,2014,6,6,5,6
35089,7,131176,771,2970,138784,70.27,5.916080,3747.544257,2.210354,158,...,7,0,4,15,2003,2014,4,8,3,2
